Task
1. Creat a RAG pipeline that can take following text and answer following questions
2. Try different types of chunking to get better answers?
3. Does asking questions differently give better answers? Why?
4. Try a different similarity search instead of cosine similarity - do the answers improve?



In [10]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re
from sklearn.metrics.pairwise import linear_kernel


In [11]:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

sample_text = """
The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.

Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter. These tribes have unique languages, traditions, and knowledge of the ecosystem. However, many face threats from illegal land encroachment and industrial activities.

Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents. The Amazon River, which flows through the rainforest, is the second longest river in the world and carries more water than any other river.

Efforts to protect the Amazon include international agreements, conservation programs, and sustainable development projects that aim to balance economic growth with environmental protection. Many organizations and governments are working to reduce illegal logging and promote reforestation initiatives.
"""

In [12]:
# Function to split text into meaningful chunks (paragraphs)
def split_text_paragraphs(text):
    return [para.strip() for para in re.split("\n+", text) if para.strip()]

# Function to split text into sentences
def split_text_sentences(text):
    return [sentence.strip() for sentence in re.split(r'(?<=[.!?]) +', text) if sentence.strip()]

# Function to split text into words
def split_text_words(text):
    return [word.strip() for word in re.split(r'\W+', text) if word.strip()]

# Store document embeddings
stored_texts_para = split_text_paragraphs(sample_text)
stored_text_sent = split_text_sentences(sample_text)
stored_text_word = split_text_words(sample_text)

stored_embeddings_p = model.encode(stored_texts_para, convert_to_numpy=True)
stored_embeddings_s = model.encode(stored_text_sent, convert_to_numpy=True)
stored_embeddings_w = model.encode(stored_text_word, convert_to_numpy=True)


In [13]:
# Function to generate embeddings
def get_transformer_embeddings(texts):
    return model.encode(texts, convert_to_numpy=True)

# Function to retrieve relevant passage
def retrieve_passage(query,s_e,s_t,cos = True):
    query_embedding = get_transformer_embeddings([query])
    if cos is True:
        similarities = cosine_similarity(query_embedding, s_e)[0]  # Ensure correct shape
    else:
        similarities = linear_kernel(query_embedding, s_e)[0]  # Ensure correct shape
    best_match_idx = np.argmax(similarities)
    return s_t[best_match_idx]

# Function to answer questions based on stored content
def answer_question(query,s_e,s_t,cos):
    relevant_passage = retrieve_passage(query,s_e,s_t,cos)
    return relevant_passage

In [15]:
questions = [
    "What is the Amazon rainforest?",
    "Which countries does the Amazon span across?",
    "Why is deforestation a problem in the Amazon?",
    "How does the Amazon rainforest affect global weather patterns?",
    "What role do indigenous tribes play in the Amazon?",
    "What is the importance of the Amazon River?",
    "What types of wildlife can be found in the Amazon?",
    "How does deforestation contribute to climate change?",
    "What efforts are being made to protect the Amazon?",
    "Why is the Amazon considered a major carbon sink?"
]
print("\nSample Questions and Answers (Paragraph Chunking with Cosine Similarity):\n")
for question in questions:
    response = answer_question(question, stored_embeddings_p, stored_texts_para, True)
    print(f"Q: {question}\nA: {response}\n")

print("\nSample Questions and Answers (Sentence Chunking with Cosine Similarity):\n")
for question in questions:
    response = answer_question(question, stored_embeddings_s, stored_text_sent, True)
    print(f"Q: {question}\nA: {response}\n")

print("\nSample Questions and Answers (Word Chunking with Cosine Similarity):\n")
for question in questions:
    response = answer_question(question, stored_embeddings_w, stored_text_word, True)
    print(f"Q: {question}\nA: {response}\n")

print("\nSample Questions and Answers (Paragraph Chunking with Linear Kernel):\n")
for question in questions:
    response = answer_question(question, stored_embeddings_p, stored_texts_para, False)
    print(f"Q: {question}\nA: {response}\n")

print("\nSample Questions and Answers (Sentence Chunking with Linear Kernel):\n")
for question in questions:
    response = answer_question(question, stored_embeddings_s, stored_text_sent, False)
    print(f"Q: {question}\nA: {response}\n")

print("\nSample Questions and Answers (Word Chunking with Linear Kernel):\n")
for question in questions:
    response = answer_question(question, stored_embeddings_w, stored_text_word, False)
    print(f"Q: {question}\nA: {response}\n")



Sample Questions and Answers (Paragraph Chunking with Cosine Similarity):

Q: What is the Amazon rainforest?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Q: Which countries does the Amazon span across?
A: The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Q: Why is deforestation a problem in the Amazon?
A: Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due 